In [1]:
# import packages
import pandas as pd
import json
#from flat_table import normalize
import ast
import re
import math

In [2]:
# load the package
df = pd.read_csv('/project/data/train_dataset.csv')
df.head()

,lens_id,jurisdiction,kind,date_published,doc_key,docdb_id,lang,biblio,families,legal_status,publication_type,abstract,claims,description
0,000-447-132-494-972,US,S,2016-10-04,US_D0768080_S_20161004,470025200.0,en,{'publication_reference': {'jurisdiction': 'US...,{'simple_family': {'members': [{'document_id':...,"{'granted': True, 'grant_date': '2016-10-04', ...",DESIGN_RIGHT,NaN,NaN,NaN
1,001-183-184-598-271,US,S,2016-11-22,US_D0772137_S_20161122,471647363.0,en,{'publication_reference': {'jurisdiction': 'US...,{'simple_family': {'members': [{'document_id':...,"{'granted': True, 'grant_date': '2016-11-22', ...",DESIGN_RIGHT,NaN,NaN,NaN
2,001-486-806-786-620,US,S,2016-11-01,US_D0770245_S_20161101,471033408.0,en,{'publication_reference': {'jurisdiction': 'US...,{'simple_family': {'members': [{'document_id':...,"{'granted': True, 'grant_date': '2016-11-01', ...",DESIGN_RIGHT,NaN,NaN,NaN
3,001-936-784-253-668,US,S,2016-11-01,US_D0770442_S_20161101,471033690.0,en,{'publication_reference': {'jurisdiction': 'US...,{'simple_family': {'members': [{'document_id':...,"{'granted': True, 'grant_date': '2016-11-01', ...",DESIGN_RIGHT,NaN,NaN,NaN
4,002-873-932-167-835,US,E,2016-11-08,US_RE046201_E_20161108,471195709.0,en,{'publication_reference': {'jurisdiction': 'US...,{'simple_family': {'members': [{'document_id':...,{'calculation_log': ['Application Filing Date:...,AMENDED_PATENT,[{'text': 'The embodiments described herein pr...,[{'claims': [{'claim_text': ['1. A method for ...,"{'text': ""BACKGROUND NAND flash memory devices..."


In [3]:
df = df.drop(columns=["families","legal_status"])

In [4]:
# convert to json
def convertJSON(series):
    series = ast.literal_eval(series)
    if type(series) is dict:
        return series
    elif type(series) is list:
        series = json.dumps(series)
        series = ast.literal_eval(series)
        return series
    else:
        None

In [5]:
# convert to json
df['biblio'] = df['biblio'].apply(convertJSON)
df['abstract'] = df['abstract'].fillna("[{'text': '', 'lang': ''}]").apply(convertJSON)
df['claims'] = df['claims'].fillna("[{'claims':[], 'lang':''}]").apply(convertJSON)

In [6]:
# drop columns
##df['families'] = df['families'].apply(convertJSON)
##df['legal_status'] = df['legal_status'].apply(convertJSON)
# df = df.drop(columns=['families'])
# df = df.drop(columns=['legal_status'])

In [7]:
df.head()

,lens_id,jurisdiction,kind,date_published,doc_key,docdb_id,lang,biblio,publication_type,abstract,claims,description
0,000-447-132-494-972,US,S,2016-10-04,US_D0768080_S_20161004,470025200.0,en,{'publication_reference': {'jurisdiction': 'US...,DESIGN_RIGHT,"[{'text': '', 'lang': ''}]","[{'claims': [], 'lang': ''}]",NaN
1,001-183-184-598-271,US,S,2016-11-22,US_D0772137_S_20161122,471647363.0,en,{'publication_reference': {'jurisdiction': 'US...,DESIGN_RIGHT,"[{'text': '', 'lang': ''}]","[{'claims': [], 'lang': ''}]",NaN
2,001-486-806-786-620,US,S,2016-11-01,US_D0770245_S_20161101,471033408.0,en,{'publication_reference': {'jurisdiction': 'US...,DESIGN_RIGHT,"[{'text': '', 'lang': ''}]","[{'claims': [], 'lang': ''}]",NaN
3,001-936-784-253-668,US,S,2016-11-01,US_D0770442_S_20161101,471033690.0,en,{'publication_reference': {'jurisdiction': 'US...,DESIGN_RIGHT,"[{'text': '', 'lang': ''}]","[{'claims': [], 'lang': ''}]",NaN
4,002-873-932-167-835,US,E,2016-11-08,US_RE046201_E_20161108,471195709.0,en,{'publication_reference': {'jurisdiction': 'US...,AMENDED_PATENT,[{'text': 'The embodiments described herein pr...,[{'claims': [{'claim_text': ['1. A method for ...,"{'text': ""BACKGROUND NAND flash memory devices..."


In [8]:
# parse a valid JSON string and convert it into a Python Dictionary
df_json= json.loads(df.to_json(orient="records"))
## print(df_json)
# normalize json
normalized_df = pd.json_normalize(df_json)

In [9]:
# parse a valid JSON string and convert it into a Python Dictionary
normalized_df_json= json.loads(normalized_df.to_json(orient="records"))
##print(normalized_df_json)

In [10]:
# Normalize semi-structured JSON data into a flat table
abstract_df = pd.json_normalize(normalized_df_json, record_path=['abstract'], meta=['lens_id', 'jurisdiction', 'kind', 'date_published', 'doc_key',
       'docdb_id', 'lang', 'description',
       'publication_type', 'biblio.publication_reference.jurisdiction',
       'biblio.publication_reference.doc_number',
       'biblio.publication_reference.kind',
       'biblio.publication_reference.date',
       'biblio.application_reference.jurisdiction',
       'biblio.application_reference.doc_number',
       'biblio.application_reference.kind',
       'biblio.application_reference.date', 'biblio.priority_claims.claims',
       'biblio.invention_title', 'biblio.parties.applicants',
       'biblio.parties.inventors', 'biblio.parties.agents',
       'biblio.parties.owners_all',
       'biblio.classifications_ipcr.classifications',
       'biblio.classifications_cpc.classifications',
       'biblio.references_cited.citations',
       'biblio.references_cited.patent_count', 'biblio.cited_by.patents',
       'biblio.cited_by.patent_count',
       'biblio.references_cited.npl_count',
       'biblio.references_cited.npl_resolved_count',
       'biblio.classifications_national.classifications'], record_prefix='abstract.')
# abstract_df.head()

In [11]:
# Normalize semi-structured JSON data into a flat table
claims_df = pd.json_normalize(normalized_df_json, record_path=['claims'], meta=['lens_id', 'jurisdiction', 'kind', 'date_published', 'doc_key',
       'docdb_id', 'lang', 'description',
       'publication_type', 'biblio.publication_reference.jurisdiction',
       'biblio.publication_reference.doc_number',
       'biblio.publication_reference.kind',
       'biblio.publication_reference.date',
       'biblio.application_reference.jurisdiction',
       'biblio.application_reference.doc_number',
       'biblio.application_reference.kind',
       'biblio.application_reference.date', 'biblio.priority_claims.claims',
       'biblio.invention_title', 'biblio.parties.applicants',
       'biblio.parties.inventors', 'biblio.parties.agents',
       'biblio.parties.owners_all',
       'biblio.classifications_ipcr.classifications',
       'biblio.classifications_cpc.classifications',
       'biblio.references_cited.citations',
       'biblio.references_cited.patent_count', 'biblio.cited_by.patents',
       'biblio.cited_by.patent_count',
       'biblio.references_cited.npl_count',
       'biblio.references_cited.npl_resolved_count',
       'biblio.classifications_national.classifications'], record_prefix='claims.')
claims_df.head()

,claims.claims,claims.lang,lens_id,jurisdiction,kind,date_published,doc_key,docdb_id,lang,description,...,biblio.parties.owners_all,biblio.classifications_ipcr.classifications,biblio.classifications_cpc.classifications,biblio.references_cited.citations,biblio.references_cited.patent_count,biblio.cited_by.patents,biblio.cited_by.patent_count,biblio.references_cited.npl_count,biblio.references_cited.npl_resolved_count,biblio.classifications_national.classifications
0,[],,000-447-132-494-972,US,S,2016-10-04,US_D0768080_S_20161004,470025200.0,en,None,...,None,None,None,None,None,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",3.0,None,None,"[{'symbol': '1303'}, {'symbol': 'D13/133'}, {'..."
1,[],,001-183-184-598-271,US,S,2016-11-22,US_D0772137_S_20161122,471647363.0,en,None,...,"[{'recorded_date': '2013-10-24', 'execution_da...",None,None,None,None,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",1.0,None,None,"[{'symbol': '1207'}, {'symbol': 'D12/345'}]"
2,[],,001-486-806-786-620,US,S,2016-11-01,US_D0770245_S_20161101,471033408.0,en,None,...,"[{'recorded_date': '2015-05-05', 'execution_da...",None,None,None,None,None,None,None,None,"[{'symbol': '0704'}, {'symbol': 'D 7672'}]"
3,[],,001-936-784-253-668,US,S,2016-11-01,US_D0770442_S_20161101,471033690.0,en,None,...,"[{'recorded_date': '2015-05-14', 'execution_da...",None,None,None,None,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",5.0,None,None,"[{'symbol': '1402'}, {'symbol': 'D14/315'}]"
4,[{'claim_text': ['1. A method for performing a...,en,002-873-932-167-835,US,E,2016-11-08,US_RE046201_E_20161108,471195709.0,en,"{'text': ""BACKGROUND NAND flash memory devices...",...,"[{'recorded_date': '2016-05-25', 'execution_da...","[{'symbol': 'G06F13/00'}, {'symbol': 'G06F12/1...",[{'symbol': 'G06F13/1668'}],"[{'sequence': 1, 'patcit': {'document_id': {'j...",106.0,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",2.0,63.0,None,None


In [12]:
# covert list to string
def listToString(series):
    claim_text = ""
    if(len(series)==0):
        return None
    for jsonObject in series:
        for x in jsonObject['claim_text']:
            claim_text += x
        claim_text += "\n"
    return claim_text

In [13]:
# merge data
merged_df = abstract_df.merge(claims_df[['claims.claims', 'lens_id', 'doc_key','docdb_id']], 
                              how='inner', on=['lens_id', 'doc_key','docdb_id'])
# merged_df = merged_df.explode('claims.claim_text')
merged_df['claims.claims']= merged_df['claims.claims'].apply(listToString)

In [14]:
merged_df.head()

,abstract.text,abstract.lang,lens_id,jurisdiction,kind,date_published,doc_key,docdb_id,lang,description,...,biblio.classifications_ipcr.classifications,biblio.classifications_cpc.classifications,biblio.references_cited.citations,biblio.references_cited.patent_count,biblio.cited_by.patents,biblio.cited_by.patent_count,biblio.references_cited.npl_count,biblio.references_cited.npl_resolved_count,biblio.classifications_national.classifications,claims.claims
0,,,000-447-132-494-972,US,S,2016-10-04,US_D0768080_S_20161004,470025200.0,en,None,...,None,None,None,None,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",3.0,None,None,"[{'symbol': '1303'}, {'symbol': 'D13/133'}, {'...",None
1,,,001-183-184-598-271,US,S,2016-11-22,US_D0772137_S_20161122,471647363.0,en,None,...,None,None,None,None,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",1.0,None,None,"[{'symbol': '1207'}, {'symbol': 'D12/345'}]",None
2,,,001-486-806-786-620,US,S,2016-11-01,US_D0770245_S_20161101,471033408.0,en,None,...,None,None,None,None,None,None,None,None,"[{'symbol': '0704'}, {'symbol': 'D 7672'}]",None
3,,,001-936-784-253-668,US,S,2016-11-01,US_D0770442_S_20161101,471033690.0,en,None,...,None,None,None,None,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",5.0,None,None,"[{'symbol': '1402'}, {'symbol': 'D14/315'}]",None
4,The embodiments described herein provide a met...,en,002-873-932-167-835,US,E,2016-11-08,US_RE046201_E_20161108,471195709.0,en,"{'text': ""BACKGROUND NAND flash memory devices...",...,"[{'symbol': 'G06F13/00'}, {'symbol': 'G06F12/1...",[{'symbol': 'G06F13/1668'}],"[{'sequence': 1, 'patcit': {'document_id': {'j...",106.0,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",2.0,63.0,None,None,1. A method for performing a sequence of comma...


In [15]:
# convert from list to string
def listToString(series):
    text = ""
    # print(type(series))
    if(len(series)==0):
        return None
    for jsonObject in series:
        keys = list(set(jsonObject.keys()))
        if(len(keys) == 1):
            key =keys[0]
            value = jsonObject[key]
            if(type(value) is list):
                for x in jsonObject[key]:
                    text += x
                text += "\n"
            elif(type(value) is str):
                text += value + ","
    return text

In [16]:
# merge data
merged_df = abstract_df.merge(claims_df[['claims.claims', 'lens_id', 'doc_key','docdb_id']], 
                              how='inner', on=['lens_id', 'doc_key','docdb_id'])
# merged_df = merged_df.explode('claims.claim_text')
merged_df['claims.claims']= merged_df['claims.claims'].apply(listToString)
merged_df['biblio.classifications_cpc.classifications'] = merged_df['biblio.classifications_cpc.classifications'].fillna("").apply(listToString)

In [17]:
merged_df.head()

,abstract.text,abstract.lang,lens_id,jurisdiction,kind,date_published,doc_key,docdb_id,lang,description,...,biblio.classifications_ipcr.classifications,biblio.classifications_cpc.classifications,biblio.references_cited.citations,biblio.references_cited.patent_count,biblio.cited_by.patents,biblio.cited_by.patent_count,biblio.references_cited.npl_count,biblio.references_cited.npl_resolved_count,biblio.classifications_national.classifications,claims.claims
0,,,000-447-132-494-972,US,S,2016-10-04,US_D0768080_S_20161004,470025200.0,en,None,...,None,None,None,None,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",3.0,None,None,"[{'symbol': '1303'}, {'symbol': 'D13/133'}, {'...",None
1,,,001-183-184-598-271,US,S,2016-11-22,US_D0772137_S_20161122,471647363.0,en,None,...,None,None,None,None,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",1.0,None,None,"[{'symbol': '1207'}, {'symbol': 'D12/345'}]",None
2,,,001-486-806-786-620,US,S,2016-11-01,US_D0770245_S_20161101,471033408.0,en,None,...,None,None,None,None,None,None,None,None,"[{'symbol': '0704'}, {'symbol': 'D 7672'}]",None
3,,,001-936-784-253-668,US,S,2016-11-01,US_D0770442_S_20161101,471033690.0,en,None,...,None,None,None,None,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",5.0,None,None,"[{'symbol': '1402'}, {'symbol': 'D14/315'}]",None
4,The embodiments described herein provide a met...,en,002-873-932-167-835,US,E,2016-11-08,US_RE046201_E_20161108,471195709.0,en,"{'text': ""BACKGROUND NAND flash memory devices...",...,"[{'symbol': 'G06F13/00'}, {'symbol': 'G06F12/1...","G06F13/1668,","[{'sequence': 1, 'patcit': {'document_id': {'j...",106.0,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",2.0,63.0,None,None,1. A method for performing a sequence of comma...


In [18]:
# drop columns
merged_df = merged_df.drop(columns=['abstract.lang','doc_key','docdb_id','lang','description',
                                   'biblio.references_cited.citations','biblio.references_cited.patent_count',
                                   'biblio.cited_by.patents','biblio.cited_by.patent_count',
                                   'biblio.references_cited.npl_count','biblio.references_cited.npl_resolved_count',
                                   'biblio.classifications_national.classifications','biblio.publication_reference.jurisdiction',
                                   'biblio.publication_reference.doc_number','biblio.publication_reference.kind',
                                   'biblio.publication_reference.date','biblio.application_reference.date',
                                   'biblio.priority_claims.claims','biblio.parties.applicants','biblio.parties.inventors',
                                   'biblio.parties.agents','biblio.parties.owners_all','biblio.application_reference.jurisdiction','biblio.application_reference.doc_number',
                                   'biblio.application_reference.kind','biblio.classifications_ipcr.classifications'])
merged_df.head()


,abstract.text,lens_id,jurisdiction,kind,date_published,publication_type,biblio.invention_title,biblio.classifications_cpc.classifications,claims.claims
0,,000-447-132-494-972,US,S,2016-10-04,DESIGN_RIGHT,"[{'text': 'Automotive plug', 'lang': 'en'}]",None,None
1,,001-183-184-598-271,US,S,2016-11-22,DESIGN_RIGHT,"[{'text': 'Tiltrotor aircraft landing gear', '...",None,None
2,,001-486-806-786-620,US,S,2016-11-01,DESIGN_RIGHT,"[{'text': 'Food slicer part', 'lang': 'en'}]",None,None
3,,001-936-784-253-668,US,S,2016-11-01,DESIGN_RIGHT,"[{'text': 'Laptop', 'lang': 'en'}]",None,None
4,The embodiments described herein provide a met...,002-873-932-167-835,US,E,2016-11-08,AMENDED_PATENT,[{'text': 'Method and controller for performin...,"G06F13/1668,",1. A method for performing a sequence of comma...


In [19]:
# Parse JSON 
merged_df_json= json.loads(merged_df.to_json(orient="records"))

In [20]:
# normalize json
final_df = pd.json_normalize(merged_df_json)
#final_df = final_df.drop(columns=['biblio.invention_title.lang'])

In [21]:
final_df.head()

,abstract.text,lens_id,jurisdiction,kind,date_published,publication_type,biblio.invention_title,biblio.classifications_cpc.classifications,claims.claims
0,,000-447-132-494-972,US,S,2016-10-04,DESIGN_RIGHT,"[{'text': 'Automotive plug', 'lang': 'en'}]",None,None
1,,001-183-184-598-271,US,S,2016-11-22,DESIGN_RIGHT,"[{'text': 'Tiltrotor aircraft landing gear', '...",None,None
2,,001-486-806-786-620,US,S,2016-11-01,DESIGN_RIGHT,"[{'text': 'Food slicer part', 'lang': 'en'}]",None,None
3,,001-936-784-253-668,US,S,2016-11-01,DESIGN_RIGHT,"[{'text': 'Laptop', 'lang': 'en'}]",None,None
4,The embodiments described herein provide a met...,002-873-932-167-835,US,E,2016-11-08,AMENDED_PATENT,[{'text': 'Method and controller for performin...,"G06F13/1668,",1. A method for performing a sequence of comma...


In [22]:
final_df.to_csv('clean_trainset.csv')